# **사전 준비**

In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "/content/drive/MyDrive/langchain-tutorial/Ch04. Advanced Rag/Data/투자설명서.pdf"
)
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap = 100)

docs = loader.load_and_split(doc_splitter)

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

# 데이터를 임베딩으로 변환
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

# FAISS 벡터 스토어 생성
faiss_store = FAISS.from_documents(docs, embedding)

# FAISS 벡터 스토어 저장
persist_directory = "/content/DB"
faiss_store.save_local(persist_directory)

# 저장한 FAISS DB 불러오기
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserializtion=True)

In [ ]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain import PromptTemplate
from typing import Literal

# **검색 필요 여부 판단 추론 파이프라인**

In [ ]:
# 출력 형식 클래스
class RetrievalResponse(BaseModel):
  Reasoning: str = Field(description="검색의 필요 여부를 추론하는 과정(2~3문장 이내)")
  Retrieve: Literal['Yes', 'No'] = Field(description="검색 필요 여부")

# 프롬프트 템플릿
retrieval_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
    주어진 질문에 대해, 외부 문서를 참고하는 것이 더 나은 응답을 생성하는 데 도움이 되는지 판단해주세요. 추론 과정을 작성한 뒤, "Yes" 또는 "No"로 답하세요

    다음 기준을 참고하세요:
    1. 사실적 정보나 복잡한 주제에 대한 상세한 설명을 요구하는 질문의 경우, 검색이 도움이 될 수 있습니다.
    2. 개인적인 의견, 창의적인 과제, 또는 간단한 계산의 경우, 일반적으로 검색이 필요하지 않습니다.
    3. 잘 알려진 사실에 대해서도, 검색은 때때로 추가적인 맥락이나 검증을 제공할 수 있습니다.

    질문: {query}
    """
)

# 사용할 LLM 설정
llm = ChatOpenAI(model="gpt-4o", max_tokens=2000, temperature=0.2)

# 각 단계에 대한 LLMChain 생성
retrieval_chain = retrieval_prompt | llm.with_structured_output(RetrievalResponse)